In [ ]:
import pandas as pd
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

In [ ]:
#Balancing dataset

df=pd.read_csv('Creditcard_data.csv')
X = df.drop('Class', axis=1)
y = df['Class']
sampler = RandomUnderSampler( random_state=42)            #undersampling
X_resampled, y_resampled = sampler.fit_resample(X, y)
print("Class Distribution in Resampled Data:")
print(y_resampled.value_counts())

# Scaling features
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)

balanced_df = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='Class')], axis=1)
print(balanced_df)

In [ ]:
#Creating 5 samples

num_samples = 5
sample_sizes = np.linspace(0.2, 1.0, num_samples)

samples = []
for size in sample_sizes:
    sample_size = int(len(balanced_df) * size)
    sample_size = max(sample_size, 1)
    sample = balanced_df.sample(n=sample_size, random_state=42)  # Random state for reproducibility
    samples.append(sample)
for i, sample in enumerate(samples):
    print(f"Sample {i+1} (Size: {len(sample)}):\n", sample.head(), "\n")


In [ ]:

models = [
    LogisticRegression(random_state=42),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42),
    GradientBoostingClassifier(random_state=42),
    SVC(random_state=42)
]

for i, sample in enumerate(samples):
    X_sample = sample
    y_sample = y_resampled[:len(sample)]
    X_sample = X_sample.drop(['Class'], axis=1)
    # Apply each model to the current sample
    for model in models:
        # Train the model
        model.fit(X_resampled_scaled, y_resampled)


        y_pred = model.predict(X_sample)
        accuracy = accuracy_score(y_sample, y_pred)
        report = classification_report(y_sample, y_pred)

        # Print results
        print(f"\nModel: {model.__class__.__name__} on Sample {i+1}")
        print(f"Accuracy: {accuracy:.4f}")
